<a href="https://colab.research.google.com/github/LukasPaulun/Theoretical-Neuroscience-2/blob/master/05%20-%20Correlations/section_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Change working directory to work in github-repo**

First, google drive has to be mounted. Do this manually or with the first two optional lines of code.

In [2]:
from google.colab import drive
drive.mount('/gdrive')

root = "/gdrive/My Drive/"
repo_name = "Theoretical-Neuroscience-2"

working_dir = root + repo_name

%cd {working_dir}

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Theoretical-Neuroscience-2


# **Pull latest changes from github and reload the neurons.py module**

In [3]:
!git fetch
!git reset --hard origin/master

%cd 'lib'

remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 100 (delta 49), reused 76 (delta 29), pack-reused 0
Receiving objects: 100% (100/100), 478.77 KiB | 11.68 MiB/s, done.
Resolving deltas: 100% (49/49), completed with 3 local objects.
From https://github.com/LukasPaulun/Theoretical-Neuroscience-2
   73af51c..504d973  master     -> origin/master
HEAD is now at 504d973 Major update on neurons.cross_correlogram().
/gdrive/My Drive/Theoretical-Neuroscience-2/lib


<module 'neurons' from '/gdrive/My Drive/Theoretical-Neuroscience-2/lib/neurons.py'>

In [6]:
import importlib
import numpy as np
import matplotlib.pyplot as plt

import neurons
import synapses
importlib.reload(neurons)
importlib.reload(synapses)

<module 'synapses' from '/gdrive/My Drive/Theoretical-Neuroscience-2/lib/synapses.py'>

# **Actual notebook starts here**

In [0]:
# Parameters
sim_time = 30
dt = 0.1e-3

# Parameters for the input populations
N_exc_pop = np.array([10, 10])
exc_target_rate = np.array([10, 10])    # [Hz]

N_inh_pop = 10
inh_target_rate = 10    # [Hz]

# Synaptic parameters
exc_init_weight = np.array([0.5, 0.5])
inh_init_weight = 1
A_P = 0.02
A_D = 0.01

# Correlation parameters
c = np.array([0.1, 0.1])

In [0]:
# Create excitatory populations and their synapses
exc_population = []
exc_synapses = []

for ii in range(N_exc_pop.size):
    # Create source neuron for population and generate source spike train
    source = neurons.PoissonNeuron(sim_time, dt)
    source.generate_spikes(exc_target_rate[ii])

    # Create excitatory population of Poisson neurons and their STDP synapses
    exc_population.append( np.array([neurons.PoissonNeuron(sim_time, dt) for _ in range(N_exc_pop[ii])]) )
    exc_synapses.append( np.array([synapses.STDPSynapse(sim_time, dt, \
                                                        init_weight=exc_init_weight[ii], \
                                                        A_P=A_P, A_D=A_D) \
                                   for _ in range(N_exc_pop[ii])]) )

    p = np.sqrt(c[ii])
    noise_rate = (1-p) * exc_target_rate[ii]
    # Generate independent Poisson spike trains and copy spikes from source neuron
    for neuron in exc_population[ii]:
        neuron.generate_spikes(noise_rate)
        neuron.copy_spikes(source, p, mode='inst')


# Create inhibitory population and synapses
inh_population = np.array([neurons.PoissonNeuron(sim_time, dt) for _ in range(N_inh_pop)])
inh_synapses = np.array([synapses.Synapse(sim_time, dt, typ='inh', init_weight=inh_init_weight) \
                         for _ in range(N_inh_pop)])
for neuron in inh_population:
    neuron.generate_spikes(inh_target_rate)

In [0]:
# Create LIF neuron and connect the two excitatory and the one inhibitory population
LIF = neurons.LIFNeuron(sim_time, dt)
LIF.connect_neurons(exc_population[0], exc_synapses[0])
LIF.connect_neurons(exc_population[1], exc_synapses[1])
LIF.connect_neurons(inh_population, inh_synapses)

In [0]:
# Run simulation
LIF.simulate()